# Creación de archivos .csv para poblar la base de datos

In [1]:
import pandas as pd
import numpy as np

import data_const as c
import utils as u
import relations as r


{'Facultad de Arquitectura': {'cp': 66455}, 'Facultad de Ciencias': {'cp': 66455}, 'Facultad de Ciencias Políticas y Sociales': {'cp': 66455}, 'Facultad de Contaduría y Administración': {'cp': 66455}, 'Facultad de Derecho': {'cp': 66455}, 'Facultad de Economía': {'cp': 66455}, 'Facultad de Filosofía y Letras': {'cp': 66455}, 'Facultad de Ingeniería': {'cp': 66455}, 'Facultad de Medicina': {'cp': 66455}, 'Facultad de Medicina Veterinaria y Zootecnia': {'cp': 66455}, 'Facultad de Odontología': {'cp': 66455}, 'Facultad de Psicología': {'cp': 66455}, 'Facultad de Química': {'cp': 66455}, 'Escuela Nacional de Trabajo Social': {'cp': 66455}, 'Escuela Nacional de Lenguas, Lingüística y Traducción': {'cp': 66455}, 'Instituto de Investigaciones en Matemáticas Aplicadas y Sistemas': {'cp': 66455}, 'Instituto de Ciencias Nucleares': {'cp': 66455}, 'Instituto de Ingeniería': {'cp': 66455}, 'Instituto de Biolgía': {'cp': 66455}, 'Instituto de Química': {'cp': 66455}, 'Instituto de Ciencias del Mar 

## Entidad Cliente

In [2]:
try:
    cliente = pd.read_csv(c.filepath_client)

except:
    print('Creando entidad Cliente')
    n_client = c.cardinality['client']
    
    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')

    names_list = u.get_names(data_names_masc, 
                             data_names_fem, 
                             data_surnames,
                            n_client,
                            1)
    # Names
    names = names_list[0]
    surnames_1 = names_list[1]
    surnames_2 = names_list[2]
    names_masc = names_list[3]
    names_fem = names_list[4]
    n_client, n_masc, n_fem = names_list[5]
    
    # Curps
    curp_masc = list(u.get_curps(names_masc,
                             surnames_1[0: n_masc],
                             surnames_2[0: n_masc],
                             sex='M'))
    curp_fem = list(u.get_curps(names_fem,
                            surnames_1[n_masc:],
                            surnames_2[n_masc:],
                            sex='F'))
    curps = curp_masc + curp_fem

    # Direcciones
    cdmx_data = pd.read_csv(c.filepath_raw + '/all_cdmx_data.csv')
    np.random.seed(1)
    to_select = np.random.choice(np.arange(0, cdmx_data.index[-1]), size=n_client, replace=True)
    cdmx_client = cdmx_data.iloc[to_select]
    cdmx_client.astype({'cp': 'float32'})
    
    unities = u.get_unity(curps)
    
    fac, ins, uni = u.get_fac_ins_uni(unities[0], unities[1], unities[2])
    
    # Email
    emails = u.get_mails(names,
                        surnames_1,
                        surnames_2,
                        curps,
                        fac,
                        ins,
                        uni)
    
    # Creamos y rellenamos
    cliente = pd.DataFrame(r.dict_to_df_cliente)

    cliente['curp'] = np.array(curps, dtype=str)
    cliente['nombre'] = np.array(names, dtype=str)
    cliente['a_paterno'] = np.array(surnames_1, dtype=str)
    cliente['a_materno'] = np.array(surnames_2, dtype=str)
    cliente['calle_num'] = np.array(cdmx_client['calle_num'], dtype=str)
    cliente['municipio'] = np.array(cdmx_client['alcaldia'], dtype=str)
    cliente['cp'] = np.array(cdmx_client['cp'], dtype=int)
    cliente['colonia'] = np.array(cdmx_client['colonia'], dtype=str)
    cliente['estado'] = str(c.address)
    cliente['tel_casa'] = np.array(u.get_tels(n_client), dtype=int)
    cliente['tel_celular'] = np.array(u.get_tels(n_client), dtype=int)
    cliente['fotografia'] = np.array(u.get_photos(curps), dtype=str)
    cliente['es_alumno'] = np.array(unities[0], dtype=bool)
    cliente['es_academico'] = np.array(unities[1], dtype=bool)
    cliente['es_trabajador'] = np.array(unities[2], dtype=bool)
    cliente['facultad'] = np.array(fac,dtype=str)
    cliente['instituto'] = np.array(ins, dtype=str)
    cliente['unidad'] = np.array(uni, dtype=str)
    cliente['email'] = np.array(emails, dtype=str)
    
    # Verificamos que todos los curps sean distintos
    assert len(set(list(cliente['curp']))) == n_client
    
    # Escribimos CSV
    
    cliente.to_csv(c.filepath_client, index=False)

In [3]:
cliente

,curp,nombre,a_paterno,a_materno,calle_num,municipio,cp,colonia,estado,tel_casa,tel_celular,email,fotografia,facultad,instituto,unidad,es_alumno,es_academico,es_trabajador
0,HEAJ810722MCYHJD86,JOSE LUIS,HEREDIA,ABAD,Angel R Cavada Mz 57 L 42,Álvaro Obregón,15000,Jalalpa Grande,Cuidad de México,5628688184,5574390078,joseheredia81@cepe.unam.mx,proyecto/data/photos/photo_HEAJ810722MCYHJD86.png,No aplica,No aplica,CEPE,False,False,True
1,COMM040711MZEISW91,ALBERTO,CONDE,MONTELLANO,Pacifico 274 Estac 14,Coyoacán,43500,Rosedal,Cuidad de México,5585615010,5673662599,albertoconde04@arquitectura.unam.mx,proyecto/data/photos/photo_COMM040711MZEISW91.png,Facultad de Arquitectura,No aplica,No aplica,True,False,False
2,GOCF010401MVJEEW81,JOSE,GOMEZ,CANSECO,"Avd. PACIFICO, 350 Dpto. E-001",Coyoacán,43300,EL ROSEDAL,Cuidad de México,5518394543,5606718628,josegomez01@arquitectura.unam.mx,proyecto/data/photos/photo_GOCF010401MVJEEW81.png,Facultad de Arquitectura,No aplica,No aplica,True,False,False
3,GALJ020515MJWEKY44,JORGE,GARCIA,LA,Cartzo 119 Local a,Cuauhtémoc,64000,Sta Maria la Rivera,Cuidad de México,5642525739,5530778735,jorgela02@psicologia.unam.mx,proyecto/data/photos/photo_GALJ020515MJWEKY44.png,Facultad de Psicología,Instituto de Biolgía,No aplica,True,True,False
4,ZUSJ031007MSAFKP77,MANUEL,ZUÑIGA,SANTOS,Camelia M-7 L-9,Álvaro Obregón,17400,San Clemente,Cuidad de México,5671754571,5543187179,manuelsantos03@ciencias.unam.mx,proyecto/data/photos/photo_ZUSJ031007MSAFKP77.jpg,Facultad de Ciencias,No aplica,No aplica,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,GACG010405FQRITY56,MARGARITA,GARCIA,CONTRERAS,Camino a Sta. Cruz Num. 53,Iztapalapa,97900,Lomas de San Lorenzo,Cuidad de México,5574091918,5603807924,margaritacontreras01@mvz.unam.mx,proyecto/data/photos/photo_GACG010405FQRITY56.jpg,Facultad de Medicina Veterinaria y Zootecnia,No aplica,No aplica,True,False,False
9996,AGDG050820FOJXQP16,GABRIELA,AGUILERA,DEOLARTE,"Prv. EMPERADOR, MZ-1 LT-38",Magdalena Contreras,10369,TIERRA UNIDA,Cuidad de México,5581272589,5523467723,gabrieladeolarte05@quimica.unam.mx,proyecto/data/photos/photo_AGDG050820FOJXQP16.jpg,Facultad de Química,No aplica,No aplica,True,False,False
9997,GUCG040522FNAYCW29,MARIA DE LOUR,GUTIERREZ,CARDENAS,Darwin 48,Miguel Hidalgo,11590,Anzures,Cuidad de México,5665840154,5604006928,mariagutierrez04@derecho.unam.mx,proyecto/data/photos/photo_GUCG040522FNAYCW29.png,Facultad de Derecho,No aplica,No aplica,True,False,False
9998,MOAG020916FHVEKI69,LAURA ESTELA,MOJICA,ALVAREZ,"2DO AND 2 DE ABRIL, MZ-5 LT-3-A",Magdalena Contreras,10900,PUEBLO SAN NICOLAS TOTOLAPAN,Cuidad de México,5508290382,5513457431,lauraalvarez02@ciencias.unam.mx,proyecto/data/photos/photo_MOAG020916FHVEKI69.jpg,Facultad de Ciencias,No aplica,No aplica,True,False,False


## Entidad Socio

In [4]:
try:
    socio = pd.read_csv(c.filepath_partner)

except:
    print('Creando entidad Socio...')
    
    n_socio = c.cardinality['partner']
    
    # nombres
    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')

    names_list = u.get_names(data_names_masc, 
                             data_names_fem, 
                             data_surnames, 
                             n_socio, 
                             2)
    
    # Names
    names = names_list[0]
    surnames_1 = names_list[1]
    surnames_2 = names_list[2]
    names_masc = names_list[3]
    names_fem = names_list[4]
    _, n_masc, n_fem = names_list[5]
    
    # Curps
    curp_masc = list(u.get_curps(names_masc,
                             surnames_1[0: n_masc],
                             surnames_2[0: n_masc],
                             sex='M', socio=True))
    curp_fem = list(u.get_curps(names_fem,
                            surnames_1[n_masc:],
                            surnames_2[n_masc:],
                            sex='F', socio=True))
    curps = curp_masc + curp_fem
    rfcs = np.array(u.get_rfc(curps), dtype=str)
    
    # Direcciones
    cdmx_data = pd.read_csv(c.filepath_raw + '/all_cdmx_data.csv')
    np.random.seed(2)
    to_select = np.random.choice(np.arange(0, cdmx_data.index[-1]), size=n_socio, replace=True)
    cdmx_socio = cdmx_data.iloc[to_select]
    cdmx_socio.astype({'cp': 'float32'})
    
    ingreso, egreso = u.get_in_out(n_socio)
    
    # Dueños y choferes
    choferes, duenios = u.get_due_cho(n_socio)
    
    # Creamos y rellenamos
    socio = pd.DataFrame(r.dict_to_df_socio)
    
    socio['id_socio'] = np.array(['s-'+f'{id}'.zfill(4) for id in range(1, n_socio+1)], dtype=str)
    socio['nombre'] = np.array(names, dtype=str)
    socio['a_paterno'] = np.array(surnames_1, dtype=str)
    socio['a_materno'] = np.array(surnames_2, dtype=str)
    socio['rfc'] = rfcs
    socio['num_licencia'] = np.array(u.get_lic(choferes), dtype=str)
    socio['calle_num'] = np.array(cdmx_socio['calle_num'], dtype=str)
    socio['municipio'] = np.array(cdmx_socio['alcaldia'], dtype=str)
    socio['cp'] = np.array(cdmx_socio['cp'], dtype=int)
    socio['colonia'] = np.array(cdmx_socio['colonia'], dtype=str)
    socio['estado'] = str(c.address)
    socio['celular'] = np.array(u.get_tels(n_socio), dtype=int)
    socio['email'] = np.array(u.get_simple_mail(names), dtype=str)
    socio['fotografia'] = np.array(u.get_photos(rfcs), dtype=str)
    socio['ingreso'] = ingreso
    socio['egreso'] = egreso
    socio['es_chofer'] = np.array(choferes, dtype=bool)
    socio['es_duenio'] = np.array(duenios, dtype=bool)
    
    # Escribimos CSV
    socio.to_csv(c.filepath_partner, index=False)
    

In [5]:
socio

,id_socio,nombre,a_paterno,a_materno,rfc,num_licencia,calle_num,colonia,cp,municipio,estado,celular,email,fotografia,ingreso,egreso,es_chofer,es_duenio
0,s-0001,LUIS ENRIQUE,FLORES,SOSA,FLSJ760701MED4,S53745449,Zapote Secc.2 Edif.centauri 139 202-a,Romulo Sanchez Mireles,14070,Tlalpan,Cuidad de México,5522353121,luis777@hotmail.com,proyecto/data/photos/photo_FLSJ760701MED4.jpg,2021-10-15,No aplica,True,True
1,s-0002,HUMBERTO,HERNANDEZ,CAMACHO,HECM820417MGH3,T05186522,Froa#o 34 B 202,Valle Gomez,15210,Venustiano Carranza,Cuidad de México,5565590412,humberto643@gmail.com,proyecto/data/photos/photo_HECM820417MGH3.jpg,2011-01-18,No aplica,True,False
2,s-0003,RODOLFO,GUERRERO,GUTIERREZ,GUGF750426MCW5,No aplica,Rio Usumacinta 33 Mza. 10 Lt. 14,Ampl. Lomas de San Bernabe,10300,Magdalena Contreras,Cuidad de México,5625717050,rodolfo709@gmail.com,proyecto/data/photos/photo_GUGF750426MCW5.jpg,2015-05-19,No aplica,False,True
3,s-0004,RUBEN,VILLANUEVA,MEZA,VIMJ801114MVE9,B28290500,Halacho Lt 13 a Mz 806,San Nicolas Totolapan,14100,Tlalpan,Cuidad de México,5520305069,ruben946@gmail.com,proyecto/data/photos/photo_VIMJ801114MVE9.jpg,2018-02-10,2018-07-30,True,False
4,s-0005,RAFAEL,SAN,GASPAR,SAGJ870217MBN6,H82224922,Edif 6 Ent D Depto 203,Unidad Lindavista Vallejo,77200,Gustavo A. Madero,Cuidad de México,5501206771,rafael610@hotmail.com,proyecto/data/photos/photo_SAGJ870217MBN6.jpg,2016-05-14,No aplica,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,s-0196,LUZ DEL CARMEN,HERNANDEZ,DOMINGUEZ,HEDG830302FUO0,No aplica,"Eje 1 PONIENTE PROL GUERRERO, 340 Edif. ALLEND...",NONOALCO TLATELOLCO,69000,Cuauhtémoc,Cuidad de México,5577105502,luz615@gmail.com,proyecto/data/photos/photo_HEDG830302FUO0.png,2014-04-11,No aplica,False,True
196,s-0197,TANIA KARINA,LOPEZ,ZAVALA,LOZD790810FID9,Q04984366,Esperanza S/n,Santo Domingo,21400,Azcapotzalco,Cuidad de México,5618593478,tania377@hotmail.com,proyecto/data/photos/photo_LOZD790810FID9.png,2010-05-19,No aplica,True,False
197,s-0198,RICARDA,RIVERA,RAMIREZ,RIRI801022FSM8,N77666450,Nezahualcoyotl Z 5 Mz 132 Lt 26,Sta Ma Aztahuacan,95000,Iztapalapa,Cuidad de México,5661081736,ricarda436@yahoo.com,proyecto/data/photos/photo_RIRI801022FSM8.png,2019-01-12,No aplica,True,False
198,s-0199,MARIA ALEJANDRA,FLORES,AGUILAR,FLAA821126FMG9,P54783204,Aguascalientes Z 1 Mz 69 Lt 4,Chalma de Guadalupe,72100,Gustavo A. Madero,Cuidad de México,5618138031,maria529@gmail.com,proyecto/data/photos/photo_FLAA821126FMG9.png,2015-02-28,No aplica,True,False


## Entidad Vehículo

In [6]:
try:
    vehicle = pd.read_csv(c.filepath_vehicle)

except:
    print('Creando entidad Vehiculo...')
    
    n_vehicle = c.cardinality['vehicle']
    poss_duenios = list(socio['id_socio'][socio.index[socio['es_duenio'] == True].tolist()])
    info_vehicles = u.get_vehicle_info(n_vehicle)
    info_aseg = u.get_aseg_info(n_vehicle)
    activos, razones = u.get_raz(n_vehicle)
    
    # Creamos y rellenamos
    vehicle = pd.DataFrame(r.dict_to_df_vehiculo)
    
    vehicle['num_economico'] = np.array(['eco-'+f'{i}'.zfill(4) for i in range(1, n_vehicle + 1)], dtype=str)
    vehicle['id_socio'] = np.array(u.get_duenios(poss_duenios, n_vehicle), dtype=str)
    vehicle['marca'] = np.array(info_vehicles[0], dtype=str)
    vehicle['modelo'] = np.array(info_vehicles[1], dtype=str)
    vehicle['anio'] = np.array(info_vehicles[2], dtype=int)
    vehicle['tipo_conduccion'] = np.array(info_vehicles[3], dtype=str)
    vehicle['num_cilindros'] = np.array(info_vehicles[4], dtype=int)
    vehicle['pasajeros'] = np.array(info_vehicles[5], dtype=int)
    vehicle['puertas'] = np.array(info_vehicles[6], dtype=int)
    vehicle['combustible'] = np.array(info_vehicles[7], dtype=str)
    vehicle['refaccion'] = np.array(info_vehicles[8], dtype=str)
    vehicle['aseguradora'] = np.array(info_aseg[0], dtype=str)
    vehicle['tipo_seguro'] = np.array(info_aseg[1], dtype=str)
    vehicle['vigencia_seg'] = info_aseg[2]
    vehicle['activo'] = np.array(activos, dtype=bool)
    vehicle['razon'] = np.array(razones, dtype=str)

    # Escribimos CSV
    vehicle.to_csv(c.filepath_vehicle, index=False)
    
    

In [7]:
vehicle

,num_economico,id_socio,marca,modelo,anio,tipo_conduccion,num_cilindros,pasajeros,puertas,combustible,refaccion,aseguradora,tipo_seguro,vigencia_seg,activo,razon
0,eco-0001,s-0020,NISSAN,Sentra,2007,Cambio automático,6,3,5,Motor gasolina,Sí,HDI,Plus,2022-10-13,True,No aplica
1,eco-0002,s-0164,NISSAN,Tsuru,2004,Cambio automático,4,4,4,Motor gasolina,Sí,GNP,Básica,2022-01-05,True,No aplica
2,eco-0003,s-0099,FORD,Figo,2012,Cambio manual,4,3,5,Motor diesel,Sí,GNP,Limitada,2021-04-05,True,No aplica
3,eco-0004,s-0003,CHEVROLET,Aveo,2020,Cambio automático,6,4,4,Motor gasolina,No,BBVA,Básica,2021-02-28,True,No aplica
4,eco-0005,s-0075,FORD,Fiesta,2013,Cambio manual,4,4,4,Motor gasolina,Sí,QUALITAS,Limitada,2021-07-21,True,No aplica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,eco-0496,s-0128,NISSAN,Tsuru,2001,Cambio automático,6,4,5,Motor gasolina,Sí,HDI,Amplia,2021-03-08,True,No aplica
496,eco-0497,s-0082,FORD,Sedán,2011,Cambio manual,6,4,5,Motor diesel,Sí,BBVA,Plus,2021-09-02,True,No aplica
497,eco-0498,s-0096,CHEVROLET,Aveo,2018,Cambio automático,4,4,5,Motor gasolina,No,GNP,Plus,2022-11-04,True,No aplica
498,eco-0499,s-0098,CHEVROLET,Aveo,2015,Cambio automático,4,4,5,Motor gasolina,Sí,BBVA,Básica,2021-05-20,True,No aplica


## Entidad Viaje

In [8]:
try:
    travel = pd.read_csv(c.filepath_travel)

except:
    print('Creando entidad Viaje...')
    
    n_travel = c.cardinality['travel']
    poss_ch = list(socio['id_socio'][socio.index[socio['es_chofer'] == True].tolist()])
    distances, times = u.get_dist_time(n_travel)
    
    # Creamos y rellenamos
    travel = pd.DataFrame(r.dict_to_df_viaje)
    
    travel['id_viaje'] = np.array(['v-' + f'{i}'.zfill(6) for i in range(1,n_travel+1)], dtype=str)
    travel['id_socio'] = np.random.choice(poss_ch, size=n_travel)
    travel['num_economico'] = np.random.choice(vehicle['num_economico'].tolist(), size=n_travel)
    travel['distancia'] = np.array(distances, dtype=float)
    travel['tiempo'] = np.array(times, dtype=int)
    travel['fecha'] = np.array([u.get_date(lower=2021)[0] for i in range(n_travel)], dtype=str)
    
    # Escribimos CSV
    travel.to_csv(c.filepath_travel, index=False)
    

In [9]:
travel

,id_viaje,id_socio,num_economico,distancia,tiempo,fecha
0,v-000001,s-0083,eco-0095,1.62,8,2021-02-15
1,v-000002,s-0068,eco-0457,1.66,11,2021-08-14
2,v-000003,s-0067,eco-0456,2.38,12,2021-07-15
3,v-000004,s-0084,eco-0322,1.45,11,2021-01-20
4,v-000005,s-0093,eco-0209,2.82,8,2021-05-24
...,...,...,...,...,...,...
24995,v-024996,s-0134,eco-0221,2.43,5,2021-04-17
24996,v-024997,s-0001,eco-0182,1.30,13,2021-04-30
24997,v-024998,s-0111,eco-0036,9.21,44,2021-08-30
24998,v-024999,s-0108,eco-0393,1.93,10,2021-05-09


## Entidad Infracción

In [12]:
try:
    infraction = pd.read_csv(c.filepath_infraction)

except:
    print('Creando entidad Infracción...')
    
    n_infraction = c.cardinality['infraction']
    infractores, nums, fechas = u.get_infrac(travel, n_infraction)
    
    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')
    oficiales_ = u.get_names(data_names_masc,
                           data_names_fem,
                           data_surnames,
                           20,
                           seed=123)
    ofi_names = oficiales_[0]
    ofi_sur1 = oficiales_[1]
    ofi_sur2 = oficiales_[2]
    oficiales = [f'{ofi_names[i]} {ofi_sur1[i]} {ofi_sur2[i]}' for i in range(20)]
    num, cp, calle = u.get_infrac_place(n_infraction)
    
    # Creamos y rellenamos
    infraction = pd.DataFrame(r.dict_to_df_infraccion)
    
    infraction['id_infraccion'] = np.array(['i-' + f'{i}'.zfill(4) for i in range(1, n_infraction+1)], dtype=str)
    infraction['id_socio'] = infractores
    infraction['num_economico'] = nums
    infraction['monto_pagar'] = [round(abs(1000.0 + np.random.normal(500, 100)), 2) for i in range(1, n_infraction+1)]
    infraction['agente'] = np.random.choice(oficiales, size=n_infraction)
    infraction['fecha'] = fechas
    infraction['hora'] = [u.get_time() for i in range(n_infraction)]
    infraction['municipio'] = num
    infraction['cp'] = cp
    infraction['calle'] = calle
    
    # Escribimos CSV
    infraction.to_csv(c.filepath_infraction, index=False)

Creando entidad Infracción...


/Users/ivanreyes/python_envs/py3.8_env/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [11]:
infraction

,id_infraccion,id_socio,num_economico,monto_pagar,agente,fecha,hora,municipio,cp,calle
0,i-0001,s-0045,eco-0235,1576.84,ISMAEL TORRES PACHECO,2021-06-28,17:56:49,Coyoacán,66455,Circuito escolar
1,i-0002,s-0068,eco-0457,1525.17,GONZALO PARRA CORTES,2021-08-14,18:30:48,Coyoacán,66455,Circuito escolar
2,i-0003,s-0168,eco-0109,1391.87,ESPERANZA LOPEZ GONZALEZ,2021-06-19,06:37:51,Coyoacán,66455,Circuito escolar
3,i-0004,s-0118,eco-0250,1564.02,ISMAEL TORRES PACHECO,2021-07-31,10:01:45,Coyoacán,66455,Circuito escolar
4,i-0005,s-0045,eco-0303,1527.74,CIRILO CRUZ BAÑAGA,2021-11-01,10:58:16,Gustavo A. Madero,73300,CALLEJON DE LA CA ADA 19
...,...,...,...,...,...,...,...,...,...,...
95,i-0096,s-0118,eco-0195,1497.46,BEATRIZ BAUTISTA SANTIAGO,2021-09-18,13:39:28,Coyoacán,66455,Circuito escolar
96,i-0097,s-0079,eco-0491,1404.27,ANA LILIA BAUTISTA RIVERA,2021-09-02,10:16:03,Coyoacán,66455,Circuito escolar
97,i-0098,s-0168,eco-0237,1444.50,ROSAURA MUÑOZ ROSAS,2021-10-13,10:34:10,Venustiano Carranza,15750,"CALZADA IGNACIO ZARAGOZA, 838 Dpto. L-003"
98,i-0099,s-0017,eco-0382,1350.32,ISMAEL TORRES PACHECO,2021-05-31,10:01:06,Coyoacán,66455,Circuito escolar
